# Connections

## Buffered Socket Connections

`proxy.py` core provides buffered socket implementations.  In most of the cases, a buffered connection will be desired.  With buffered connections, we can queue data from our application code while leaving the responsibility of flushing the buffer on the core.

One of the buffered connection class is `TcpServerConnection`, which manages connection to an upstream server.  Optionally, we can also enable encryption _(TLS)_ before communicating with the server.

Import the following:

In [ ]:
from proxy.core.connection import TcpServerConnection
from proxy.common.utils import build_http_request
from proxy.http.methods import httpMethods
from proxy.http.parser import HttpParser, httpParserTypes

Let's use `TcpServerConnection` to make a HTTP web server request.

In [ ]:
request = build_http_request(
    method=httpMethods.GET,
    url=b'/',
    headers={
        b'Host': b'jaxl.com',
    }
)

http_client = TcpServerConnection('jaxl.com', 80)
http_client.connect()
http_client.queue(memoryview(request))
http_client.flush()

http_response = HttpParser(httpParserTypes.RESPONSE_PARSER)
while not http_response.is_complete:
    http_response.parse(http_client.recv().tobytes())
http_client.close()

print(http_response.build_response())

assert http_response.is_complete
assert http_response.code == b'301'
assert http_response.reason == b'Moved Permanently'
assert http_response.has_header(b'location')
assert http_response.header(b'location') == b'https://jaxl.com/'

Let's use `TcpServerConnection` to make a HTTPS web server request.

In [ ]:
https_client = TcpServerConnection('jaxl.com', 443)
https_client.connect()
https_client.wrap(hostname='jaxl.com', as_non_blocking=False)

https_client.queue(memoryview(request))
https_client.flush()

https_response = HttpParser(httpParserTypes.RESPONSE_PARSER)
while not https_response.is_complete:
    https_response.parse(https_client.recv().tobytes())
https_client.close()

print(https_response.build_response())

assert https_response.is_complete
assert https_response.code == b'200'
assert https_response.reason == b'OK'
assert https_response.has_header(b'content-type')
assert https_response.header(b'content-type') == b'text/html'